# Basic Backtest

This notebook demonstrates how to run a simple backtest using the trading system.

## What is a Backtest?

A backtest simulates trading a strategy on historical data to evaluate its performance. The system:
- Processes data day-by-day (no lookahead bias)
- Generates entry/exit signals based on strategy rules
- Simulates realistic execution (slippage, fees)
- Tracks portfolio performance over time


In [ ]:
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Add project root to path
project_root = Path().resolve().parent.parent.parent
sys.path.insert(0, str(project_root))

from trading_system.integration.runner import run_backtest
from trading_system.configs.run_config import RunConfig
from trading_system.reporting.metrics import MetricsCalculator

print("Setup complete!")


In [ ]:
# Load run configuration
config_path = project_root / "tests" / "fixtures" / "configs" / "run_test_config.yaml"

if not config_path.exists():
    print(f"Config not found: {config_path}")
    print("Using example config instead...")
    config_path = project_root / "EXAMPLE_CONFIGS" / "run_config.yaml"

print(f"Loading config: {config_path}")
run_config = RunConfig.from_yaml(str(config_path))

print(f"\nConfiguration loaded:")
print(f"  Start date: {run_config.dataset.start_date}")
print(f"  End date: {run_config.dataset.end_date}")
print(f"  Starting equity: ${run_config.portfolio.starting_equity:,.0f}")


In [ ]:
# Run backtest on training period
print("Running backtest...")
print("This may take a few moments...")

results = run_backtest(
    config_path=str(config_path),
    period="train"  # Options: "train", "validation", "holdout"
)

print("\nBacktest complete!")
print(f"Run ID: {results.run_id}")


In [ ]:
# Calculate metrics
metrics_calc = MetricsCalculator()
metrics = metrics_calc.calculate_all(
    equity_curve=results.equity_curve,
    trades=results.trades,
    benchmark_returns=results.benchmark_returns
)

print("=== Backtest Results ===")
print(f"\nPeriod: {results.period}")
print(f"Start Date: {results.start_date}")
print(f"End Date: {results.end_date}")
print(f"\n--- Performance Metrics ---")
print(f"Total Return: {metrics['total_return']*100:.2f}%")
print(f"Annualized Return: {metrics['annualized_return']*100:.2f}%")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
print(f"Max Drawdown: {metrics['max_drawdown']*100:.2f}%")
print(f"Calmar Ratio: {metrics['calmar_ratio']:.2f}")
print(f"\n--- Trade Statistics ---")
print(f"Total Trades: {metrics['total_trades']}")
print(f"Winning Trades: {metrics['winning_trades']}")
print(f"Losing Trades: {metrics['losing_trades']}")
print(f"Win Rate: {metrics['win_rate']*100:.2f}%")
print(f"Average Win: {metrics['avg_win']*100:.2f}%")
print(f"Average Loss: {metrics['avg_loss']*100:.2f}%")
print(f"Profit Factor: {metrics['profit_factor']:.2f}")
